In [ ]:
# -*- coding: utf-8 -*-
"""
Jupyter Notebook 版本：多轮对话 + KV 缓存 + Markdown/LaTeX 渲染
- 用 IPython.display.Markdown 渲染答案（支持 MathJax 公式）
- DynamicCache 复用 KV，前缀对齐增量更新
- 人工输入循环（在 Notebook 里 input() 可用）
- 可选显示 <think> 内容
"""

import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import logging as hf_logging
from transformers.cache_utils import DynamicCache
from IPython.display import display, Markdown, Latex

# 日志更安静
hf_logging.set_verbosity_warning()

# ===== 配置 =====
model_name = "qw1.7_model"   # TODO: 改成你的模型
max_new_tokens = 2048 
use_sampling = False          # True=采样; False=贪心
temperature = 0.7
top_p = 0.9
show_think = False            # True 显示 <think> 内容

# ===== 加载模型/分词器 =====
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)
model.eval()
device = model.device

# 兜底 pad_token
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# ===== 初始对话历史 =====
messages = [
    {"role": "system", "content": "你是‘地震学AI助手’，一个乐于助人的中文AI助手。由地震局人工智能团队开发。请帮助用户解决问题。"}, 
]

# ===== KV 缓存状态 =====
cached_input_ids = None        # torch.LongTensor [1, L]
kv_cache: DynamicCache | None = None

def build_input_ids_from_messages(msgs):
    """用聊天模板把 messages 转成 token 序列"""
    prompt = tokenizer.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True  # 让模型继续生成 assistant 的回答
    )
    return tokenizer(prompt, return_tensors="pt").to(device)["input_ids"]

@torch.no_grad()
def prefill_or_update_cache(new_input_ids):
    """
    复用 KV 的关键：
    - 第一次/缓存失效：全量预填（prefill）
    - 否则若新输入以旧输入为前缀，仅对“新增 token”做前向，增量更新 KV
    - 不匹配则重建 KV
    """
    global cached_input_ids, kv_cache

    # 第一次/缓存失效
    if cached_input_ids is None or kv_cache is None:
        kv_cache = DynamicCache()
        _ = model(input_ids=new_input_ids, use_cache=True, past_key_values=kv_cache)
        cached_input_ids = new_input_ids
        last_logits = model(input_ids=new_input_ids[:, -1:], use_cache=True, past_key_values=kv_cache).logits[:, -1, :]
        return last_logits

    old_len = cached_input_ids.shape[1]
    # 前缀对齐：只跑新增 token
    if new_input_ids.shape[1] >= old_len and torch.equal(new_input_ids[:, :old_len], cached_input_ids):
        delta = new_input_ids[:, old_len:]
        if delta.numel() > 0:
            _ = model(input_ids=delta, use_cache=True, past_key_values=kv_cache)
            cached_input_ids = new_input_ids
        last_logits = model(input_ids=new_input_ids[:, -1:], use_cache=True, past_key_values=kv_cache).logits[:, -1, :]
        return last_logits

    # 不匹配：重建 KV
    kv_cache = DynamicCache()
    _ = model(input_ids=new_input_ids, use_cache=True, past_key_values=kv_cache)
    cached_input_ids = new_input_ids
    last_logits = model(input_ids=new_input_ids[:, -1:], use_cache=True, past_key_values=kv_cache).logits[:, -1, :]
    return last_logits

@torch.no_grad()
def generate_with_kv(last_logits, max_tokens=512, eos_id=None):
    """
    显式逐 token 生成，完全依赖 KV（每步只喂一个新 token）
    可切换采样/贪心；避免使用 model.generate 产生参数兼容性日志
    """
    global kv_cache
    generated_ids = []
    cur_logits = last_logits

    for _ in range(max_tokens):
        if use_sampling:
            probs = torch.softmax(cur_logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)  # [1,1]
        else:
            next_id = torch.argmax(cur_logits, dim=-1, keepdim=True)  # [1,1]

        tid = next_id.item()
        generated_ids.append(tid)

        if eos_id is not None and tid == eos_id:
            break

        # 增量一步：只喂入新 token，更新 KV 并取下一个位置 logits
        cur_logits = model(input_ids=next_id, use_cache=True, past_key_values=kv_cache).logits[:, -1, :]

    if len(generated_ids) == 0:
        return torch.empty((1, 0), dtype=torch.long, device=device)
    return torch.tensor(generated_ids, dtype=torch.long, device=device).view(1, -1)

def split_think(text: str):
    """抽出 <think> 内容，并返回外层文本"""
    insides = re.findall(r"<think>(.*?)</think>", text, flags=re.S)
    outside = re.sub(r"<think>.*?</think>", "", text, flags=re.S).strip()
    return insides, outside

def render_markdown(md_text: str, title: str | None = None):
    """
    在 Jupyter 中用 Markdown 渲染文本（自动支持 LaTeX 公式）
    - 行内：$...$
    - 块级：$$...$$
    """
    if title:
        display(Markdown(f"### {title}"))
    display(Markdown(md_text))

# ======== 首轮对话（演示） ========
#input_ids = build_input_ids_from_messages(messages)
#last_logits = prefill_or_update_cache(input_ids)
#gen_ids = generate_with_kv(last_logits, max_tokens=max_new_tokens, eos_id=tokenizer.eos_token_id)
#
#full_ids = torch.cat([input_ids, gen_ids], dim=1)
#text = tokenizer.decode(full_ids[0], skip_special_tokens=True)
#
#think_spans, answer = split_think(text)
#final_answer = answer.split("assistant")[-1].strip()
#
#if show_think and think_spans:
#    display(Markdown("**思考过程 `<think>`：**"))
#    display(Markdown(f"```\n{think_spans[-1].strip()}\n```"))
#
#render_markdown(final_answer, title="助手回复（Markdown/LaTeX 渲染）")
#
## 写回历史 + 并入缓存
#messages.append({"role": "assistant", "content": final_answer})
#cached_input_ids = full_ids

# ======== 多轮对话循环（在 Notebook 中人工输入） ========
print("进入多轮对话（输入 q 退出）")
while True:
    try:
        user_text = input("\n你：").strip()
        if user_text.lower() in {"q", "quit", "exit"}:
            print("已退出。")
            break
        if not user_text:
            continue

        messages.append({"role": "user", "content": user_text})

        # 构造输入 + 复用 KV
        input_ids = build_input_ids_from_messages(messages)
        last_logits = prefill_or_update_cache(input_ids)
        gen_ids = generate_with_kv(last_logits, max_tokens=max_new_tokens, eos_id=tokenizer.eos_token_id)

        # 解码 + 渲染
        full_ids = torch.cat([input_ids, gen_ids], dim=1)
        text = tokenizer.decode(full_ids[0], skip_special_tokens=True)
        think_spans, answer = split_think(text)
        final_answer = answer.split("assistant")[-1].strip()

        if show_think and think_spans:
            display(Markdown("**思考过程 `<think>`：**"))
            display(Markdown(f"```\n{think_spans[-1].strip()}\n```"))

        render_markdown(final_answer, title="助手回复")

        # 写回历史 + 并入缓存
        messages.append({"role": "assistant", "content": final_answer})
        cached_input_ids = torch.cat([input_ids, gen_ids], dim=1)

    except KeyboardInterrupt:
        print("\n已退出。")
        break


In [ ]:
# -*- coding: utf-8 -*-
"""
Jupyter Notebook：多轮对话 + KV 缓存 + Markdown/LaTeX 流式渲染
- 人工输入多轮对话
- DynamicCache 复用 KV（前缀对齐增量预填）
- 每生成一部分 token 就流式更新显示（MathJax 渲染公式）
"""

import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import logging as hf_logging
from transformers.cache_utils import DynamicCache
from IPython.display import display, Markdown

hf_logging.set_verbosity_warning()

# ===== 配置 =====
model_name = "qw32_model"     # ← 改成你的模型名称或本地路径
max_new_tokens = 2048 
stream_interval = 8            # 每多少个 token 刷新一次
use_sampling = False           # True=采样；False=贪心
temperature = 0.7
top_p = 0.9
show_think = False             # 是否显示 <think> 内容

# ===== 加载模型/分词器 =====
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)
model.eval()
device = model.device

# 兜底 pad_token
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# ===== 对话历史（可根据需要预置开场） =====
messages = [
    {"role": "system", "content": "你是‘地震学AI助手’，一个乐于助人的中文AI助手。由地震局人工智能团队开发。请帮助用户解决问题。"},
]

# ===== KV 缓存状态 =====
cached_input_ids = None        # torch.LongTensor [1, L]
kv_cache: DynamicCache | None = None

def build_input_ids_from_messages(msgs):
    """将 messages 用聊天模板转成 token 序列"""
    prompt = tokenizer.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True
    )
    return tokenizer(prompt, return_tensors="pt").to(device)["input_ids"]

@torch.no_grad()
def prefill_or_update_cache(new_input_ids):
    """
    复用 KV 的关键：
    - 第一次/缓存失效：全量预填（prefill）
    - 否则若新输入以旧输入为前缀：仅对“新增 token”做前向，增量更新 KV
    - 不匹配则重建 KV
    """
    global cached_input_ids, kv_cache

    # 第一次/缓存失效
    if cached_input_ids is None or kv_cache is None:
        kv_cache = DynamicCache()
        _ = model(input_ids=new_input_ids, use_cache=True, past_key_values=kv_cache)
        cached_input_ids = new_input_ids
        last_logits = model(input_ids=new_input_ids[:, -1:], use_cache=True, past_key_values=kv_cache).logits[:, -1, :]
        return last_logits

    old_len = cached_input_ids.shape[1]
    # 前缀对齐：只跑新增 token
    if new_input_ids.shape[1] >= old_len and torch.equal(new_input_ids[:, :old_len], cached_input_ids):
        delta = new_input_ids[:, old_len:]
        if delta.numel() > 0:
            _ = model(input_ids=delta, use_cache=True, past_key_values=kv_cache)
            cached_input_ids = new_input_ids
        last_logits = model(input_ids=new_input_ids[:, -1:], use_cache=True, past_key_values=kv_cache).logits[:, -1, :]
        return last_logits

    # 不匹配：重建 KV
    kv_cache = DynamicCache()
    _ = model(input_ids=new_input_ids, use_cache=True, past_key_values=kv_cache)
    cached_input_ids = new_input_ids
    last_logits = model(input_ids=new_input_ids[:, -1:], use_cache=True, past_key_values=kv_cache).logits[:, -1, :]
    return last_logits

def split_think(text: str):
    """抽取 <think> 内容，并返回外层文本"""
    insides = re.findall(r"<think>(.*?)</think>", text, flags=re.S)
    outside = re.sub(r"<think>.*?</think>", "", text, flags=re.S).strip()
    return insides, outside

@torch.no_grad()
def stream_generate(last_logits, eos_id=None, max_tokens=512, base_prompt_ids=None, display_id=None):
    """
    逐 token 生成 + 流式渲染（在 Notebook 中持续更新同一个输出块）
    - base_prompt_ids: prompt 的 token（用于 decode 完整上下文避免分词裂缝）
    - display_id: Notebook display_id（复用同一个显示块）
    返回：本轮生成的 token ids (tensor [1, T])
    """
    global kv_cache
    generated_ids = []
    cur_logits = last_logits

    # 用 display_id 创建/获取一个输出块
    if display_id is None:
        handle = display(Markdown(""), display_id=True)
    else:
        handle = display(Markdown(""), display_id=display_id)

    for step in range(max_tokens):
        # 选 token
        if use_sampling:
            probs = torch.softmax(cur_logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
        else:
            next_id = torch.argmax(cur_logits, dim=-1, keepdim=True)

        tid = next_id.item()
        generated_ids.append(tid)

        # 结束符
        if eos_id is not None and tid == eos_id:
            break

        # 每隔若干 token 刷新一次（把上下文+已生成一起 decode，保证 Markdown/公式完整性尽量好）
        if (step + 1) % stream_interval == 0:
            if base_prompt_ids is not None:
                full_ids = torch.cat([base_prompt_ids, torch.tensor(generated_ids, device=device).view(1, -1)], dim=1)
                text = tokenizer.decode(full_ids[0], skip_special_tokens=True)
            else:
                text = tokenizer.decode(generated_ids, skip_special_tokens=True)

            _, outside = split_think(text)
            partial_answer = outside.split("assistant")[-1].strip()
            handle.update(Markdown(partial_answer))

        # 增量一步：只喂入新 token，更新 KV，并取下一个位置 logits
        cur_logits = model(input_ids=next_id, use_cache=True, past_key_values=kv_cache).logits[:, -1, :]

    # 最终完整输出
    if generated_ids:
        if base_prompt_ids is not None:
            full_ids = torch.cat([base_prompt_ids, torch.tensor(generated_ids, device=device).view(1, -1)], dim=1)
            text = tokenizer.decode(full_ids[0], skip_special_tokens=True)
        else:
            text = tokenizer.decode(generated_ids, skip_special_tokens=True)

        think_spans, outside = split_think(text)
        final_answer = outside.split("assistant")[-1].strip()

        if show_think and think_spans:
            # 先显示 think，再显示最终答案
            handle.update(Markdown("**思考过程 `<think>`：**\n\n```\n" + think_spans[-1].strip() + "\n```"))
            handle = display(Markdown(""), display_id=True)

        handle.update(Markdown(final_answer))

    if len(generated_ids) == 0:
        return torch.empty((1, 0), dtype=torch.long, device=device)
    return torch.tensor(generated_ids, dtype=torch.long, device=device).view(1, -1)

# ===== 多轮对话循环 =====
print("进入多轮对话（在下方输入，'q' 退出）")
while True:
    try:
        user_text = input("\n你：").strip()
        if user_text.lower() in {"q", "quit", "exit"}:
            print("已退出。")
            break
        if not user_text:
            continue

        # 追加用户消息
        messages.append({"role": "user", "content": user_text})

        # 构造输入 + 复用 KV（仅增量预填）
        input_ids = build_input_ids_from_messages(messages)
        last_logits = prefill_or_update_cache(input_ids)

        # 启动流式生成：把当前 prompt ids 传入，用于流式 decode 完整上下文
        gen_ids = stream_generate(
            last_logits,
            eos_id=tokenizer.eos_token_id,
            max_tokens=max_new_tokens,
            base_prompt_ids=input_ids,
        )

        # 取最终答案文本（用于写回历史 & 下轮前缀）
        full_ids = torch.cat([input_ids, gen_ids], dim=1)
        text = tokenizer.decode(full_ids[0], skip_special_tokens=True)
        _, outside = split_think(text)
        final_answer = outside.split("assistant")[-1].strip()

        # 写回历史；合并到缓存前缀
        messages.append({"role": "assistant", "content": final_answer})
        cached_input_ids = full_ids

    except KeyboardInterrupt:
        print("\n已退出。")
        break


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

进入多轮对话（在下方输入，'q' 退出）


好的，用户让我解释地震面波。首先，我需要确定用户对地震学的基础知识了解多少。可能他们之前接触过地震波的基本分类，比如体波和面波，但需要更详细的解释。或者他们可能完全不了解，需要从头开始讲起。

接下来，我应该先定义什么是地震面波。面波是沿着地球表面传播的地震波，和体波不同，体波是在地球内部传播的。面波通常在地震发生后到达，速度比体波慢，但振幅更大，所以破坏力更强。这一点很重要，因为用户可能想知道为什么面波更危险。

然后，我需要分类型讲解。面波主要有两种：瑞利波和洛夫波。瑞利波的运动方式像海浪，既有垂直也有水平运动，而洛夫波主要是水平方向的剪切运动。这部分需要详细说明两者的区别，可能用户会混淆这两种波的运动方式。

还要提到面波的传播路径和速度。面波只在地表附近传播，速度比体波慢，所以到达时间更晚。但因为它们的能量集中在地表，所以对建筑物的破坏更大。这里可能需要举例说明，比如在地震中，面波导致的摇晃更持久，更容易造成结构损坏。

另外，用户可能想知道面波在地震学中的应用。比如，面波用于研究地球内部结构，或者在地震预警系统中的作用。这部分可以简要提及，但不需要太深入，除非用户进一步询问。

还要注意术语的解释是否清晰，比如“振幅”、“剪切运动”等，可能需要用更通俗的语言。同时，检查是否有常见的误解需要澄清，比如面波是否总是比体波慢，或者是否所有地震都有面波。

最后，确保回答结构清晰，分点说明，让用户容易理解。可能需要先总述面波的定义和特点，再分类型详细解释，最后总结其影响和应用。同时，保持语言简洁，避免过于专业的术语，除非用户有相关背景。
</think>

地震面波是地震波的一种，主要沿地球表面传播，是地震发生后到达地表的次生波。与体波（如P波和S波）不同，面波的能量集中在地表附近，因此对地表建筑物和人类活动的影响更为显著。以下是关于地震面波的详细解释：

---

### **1. 面波的类型**
面波主要分为两种类型：
- **瑞利波（Rayleigh Wave）**  
  - **运动方式**：类似海浪的滚动运动，既有垂直方向的上下振动，也有水平方向的前后摆动。  
  - **特点**：振幅较大，传播速度较慢（约2.0-4.5 km/s），是地震中常见的破坏性波。  
  - **破坏性**：由于其复杂的运动模式，容易引发建筑物的共振，导致更严重的破坏。

- **洛夫波（Love Wave）**  
  - **运动方式**：仅在水平方向上横向剪切运动（类似S波的剪切效应）。  
  - **特点**：传播速度略快于瑞利波（约2.5-5.0 km/s），但振幅通常较小。  
  - **破坏性**：对地表结构（如桥梁、管道）的水平剪切破坏较强。

---

### **2. 面波的传播特性**
- **传播路径**：面波仅沿地球表面传播，能量集中在地表几公里范围内。  
- **速度**：比体波（P波和S波）慢，因此在地震中